In [1]:
import cv2
import os
import time
import numpy as np
import mediapipe as mp
from matplotlib import pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Data Collection

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert color
    image.flags.writeable = False # No longer writeable
    results = model.process(image) # Make prediction
    image.flags.writeable = True # Writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Convert color back
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # hand left
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # hand right

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    #(0,0,250) - > BGR
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                             )

In [20]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
    
        # Make predicitons
        image, results = mediapipe_detection(frame, holistic)
        #print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
results.pose_landmarks

In [6]:
# Concatenate all the different landmarks into a numpy array which will be used in our LSTM model.
# Replace numpy array with zeroes if there aren't any landmarks.
def extract_keypoints(results):
    #import sys
    #np.set_printoptions(threshold=sys.maxsize) - To print all results
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_hand, right_hand])

In [ ]:
result_keypoints = extract_keypoints(results)

In [ ]:
result_keypoints

In [13]:
np.save('0', result_keypoints)

In [10]:
# Used to setup folders for traning the LSTM model

# Path for exported Data, numpy arrays
DATA_PATH = os.path.join('MP_DATA') 

# Actions we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [ ]:
# Create one folder for each action
for action in actions:
    for sequence in range(no_sequences):
        try:
            new_path = os.path.join(DATA_PATH, action, str(sequence))
            os.makedirs(new_path)
        except OSError as error:
            print("Directory '%s' can not be created")

In [9]:
# Collecting Data
# 1h06m51s
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop Through actions
    for action in actions:
        # loop through sequences
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
            
                # Make predicitons
                image, results = mediapipe_detection(frame, holistic)
                #print(results)
        
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frame for {} video number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    # Show on screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frame for {} video number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

                # Show on screen
                cv2.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

# Preprocessing Data and Creating labels and features

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert your data into one-hot encoding

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
np.array(labels).shape

(90,)

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(90, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [20]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [21]:
y_test.shape

(5, 3)

# LSTM NN using Tensorflow and Keras

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
# Used to limit GPU memory growth & make sure Tensorflow uses my gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) # Input layer
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # Output Layer

In [45]:
res = [0.7, 0.2, 0.1]

In [46]:
actions[np.argmax(res)]

'hello'

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) # categorical_crossentropy - for multiclass classification
# Binary_crossentropy - Binary classification

In [ ]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# Make predictions

In [33]:
model_result = model.predict(X_test)

1/1 [==============================] - 0s 337ms/step


In [37]:
actions[np.argmax(model_result[2])]

'iloveyou'

In [36]:
actions[np.argmax(y_test[2])]

'iloveyou'

# Save weights

In [38]:
OPTIMIZER = 'Adam'
EPOCHS = 150
LOSS ='categorical_crossentropy'
LAYER_COUNT = 6
MODEL_NAME = 'Sign Detection Model - Optimizer {} - Epochs {} - Loss {} - Layers {}'.format(OPTIMIZER, str(EPOCHS), LOSS, str(LAYER_COUNT))
model.save('{}.h5'.format(MODEL_NAME))

In [ ]:
# Evaluation with Confusion Matrix and Accuracy

In [28]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [31]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 897ms/step


In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
yhat

[2, 2, 2, 2, 1]

In [34]:
multilabel_confusion_matrix(ytrue, yhat) # We want our values in to left and bottom right 

array([[[3, 0],
        [2, 0]],

       [[4, 0],
        [0, 1]],

       [[1, 2],
        [0, 2]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

0.6

In [29]:
model.load_weights('Sign Detection Model - Optimizer Adam - Epochs 150 - Loss categorical_crossentropy - Layers 6.h5')

# Test In Real-Time

In [37]:
sequence = [] # 30 frames
sentence = [] # Concat
threshold = 0.4 # confidence metrics
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3, model_complexity=2) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Make Prediction
        image, results = mediapipe_detection(frame, holistic)
        #print(resultVid)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Newly added
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        # Visual logic
        if res[np.argmax(res)] > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1:]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 42ms/step
thanks
1/1 [==============================] - 0s 60ms/step
thanks
1/1 [==============================] - 0s 41ms/step
thanks
1/1 [==============================] - 0s 65ms/step
thanks
1/1 [==============================] - 0s 39ms/step
thanks
1/1 [==============================] - 0s 46ms/step
thanks
1/1 [==============================] - 0s 42ms/step
thanks
1/1 [==============================] - 0s 47ms/step
iloveyou
1/1 [==============================] - 0s 42ms/step
iloveyou
1/1 [==============================] - 0s 41ms/step
iloveyou
1/1 [==============================] - 0s 41ms/step
iloveyou
1/1 [==============================] - 0s 76ms/step
iloveyou
1/1 [==============================] - 0s 50ms/step
iloveyou
1/1 [==============================] - 0s 64ms/step
iloveyou
1/1 [==============================] - 0s 41ms/step
iloveyou
1/1 [==============================] - 0s 41ms/step
iloveyou
1/1 [==============================] -